# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

In [1]:
import pandas as pd
addres_book = pd.read_json('data/addres-book.json', orient='records')
# print(addres_book)
print(addres_book[:]['email'])

0     faina@mail.ru
1    robert@mail.ru
Name: email, dtype: object


In [2]:
import json
with open('data/addres-book.json', 'r', encoding='utf-8') as f:
    addres_book_2 = json.load(f)
# print(addres_book_2)
for item in addres_book_2:
    print(item['email'])

faina@mail.ru
robert@mail.ru


2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

In [3]:
for phones in addres_book['phones']:
    for phone in phones:
        print(phone['phone'])

232-19-55
+7 (916) 232-19-55
111-19-55
+7 (916) 445-19-55


In [4]:
for item in addres_book_2:
    for phone in item['phones']:
        print(phone['phone'])

232-19-55
+7 (916) 232-19-55
111-19-55
+7 (916) 445-19-55


3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

In [5]:
addres_book_q = pd.read_xml('data/addres-book-q.xml')
addres_book_q

,name,address
0,algeria,NaN
1,angola,NaN
2,argentina,NaN
3,australia,NaN


## Лабораторная работа №3

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [6]:
import json
with open('data/contributors_sample.json', 'r', encoding='utf-8') as f:
    contributors_sample = json.load(f)
contributors_sample[:3]

[{'username': 'uhebert',
  'name': 'Lindsey Nguyen',
  'sex': 'F',
  'address': '01261 Cameron Spring\nTaylorfurt, AK 97791',
  'mail': 'jsalazar@gmail.com',
  'jobs': ['Energy engineer',
   'Engineer, site',
   'Environmental health practitioner',
   'Biomedical scientist',
   'Jewellery designer'],
  'id': 35193},
 {'username': 'vickitaylor',
  'name': 'Cheryl Lewis',
  'sex': 'F',
  'address': '66992 Welch Brooks\nMarshallshire, ID 56004',
  'mail': 'bhudson@gmail.com',
  'jobs': ['Music therapist',
   'Volunteer coordinator',
   'Designer, interior/spatial'],
  'id': 91970},
 {'username': 'sheilaadams',
  'name': 'Julia Allen',
  'sex': 'F',
  'address': 'Unit 1632 Box 2971\nDPO AE 23297',
  'mail': 'darren44@yahoo.com',
  'jobs': ['Management consultant',
   'Engineer, structural',
   'Lecturer, higher education',
   'Theatre manager',
   'Designer, textile'],
  'id': 1848091}]

1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [7]:
{contributor['mail'].split('@')[1] for contributor in contributors_sample}

{'gmail.com', 'hotmail.com', 'yahoo.com'}

1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [8]:
def find_username(username):
    for contributor in contributors_sample:
        if contributor['username'] == username:
            return contributor
    else:
        raise ValueError('wrong username: ' + username)
username = input()
print(find_username(username))

sheilaadams
{'username': 'sheilaadams', 'name': 'Julia Allen', 'sex': 'F', 'address': 'Unit 1632 Box 2971\nDPO AE 23297', 'mail': 'darren44@yahoo.com', 'jobs': ['Management consultant', 'Engineer, structural', 'Lecturer, higher education', 'Theatre manager', 'Designer, textile'], 'id': 1848091}


1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [9]:
f, m = 0, 0
for contributor in contributors_sample:
    if contributor['sex'] == 'F':
        f += 1
    else:
        m += 1
print('Female users:', f)
print('Male users:', m)

Female users: 2136
Male users: 2064


1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [10]:
contributors = pd.read_json('data/contributors_sample.json', orient='records')[['id', 'username', 'sex']]
contributors

,id,username,sex
0,35193,uhebert,F
1,91970,vickitaylor,F
2,1848091,sheilaadams,F
3,50969,nicole82,F
4,676820,jean67,M
...,...,...,...
4195,423555,stevenspencer,F
4196,35251,rwilliams,M
4197,135887,lmartinez,F
4198,212714,brendahill,M


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [11]:
recipes = pd.read_csv('data/recipes_sample.csv', parse_dates=['submitted'])
merged = pd.merge(recipes, contributors, left_on='contributor_id', right_on='id', how='left')
len(merged[merged['id_y'].isnull()])

15059

### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [12]:
with open('data/contributors_sample.json', 'r', encoding='utf-8') as f: # открываем файл на чтение
    contributors_sample = json.load(f)
job_people = {}
for contributor in contributors_sample:
    for job in contributor['jobs']:
        job_people[job] = job_people.get(job, []) + [contributor['username']]

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [13]:
import pickle
with open('data/job_people.pickle', 'wb') as f:
    pickle.dump(job_people, f)

In [14]:
with open('data/job_people.json', mode='w', encoding='utf-8') as f:
    json.dump(job_people, f, indent=2)

2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [15]:
with open('data/job_people.pickle', 'rb') as f:
    job_pickle = pickle.load(f)
# job_pickle

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [16]:
from bs4 import BeautifulSoup
with open('data/steps_sample.xml') as f:
    steps_sample = BeautifulSoup(f, 'xml')

In [17]:
steps_dict = {}
for recipe in steps_sample.recipes.find_all('recipe'):
    steps = [step.text for step in recipe.steps.find_all('step')]
    steps_dict[int(recipe.id.text)] = steps

In [18]:
with open('data/steps_sample.json', mode='w', encoding='utf-8') as f:
    json.dump(steps_dict, f, indent=2)

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [19]:
steps_num = {}
for recipe in steps_sample.recipes.find_all('recipe'):
    num = len(recipe.steps.find_all('step'))
    steps_num[num] = steps_num.get(num, []) + [int(recipe.id.text)] 
# steps_num

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [20]:
recipes_time = []
for recipe in steps_sample.recipes.find_all('recipe'):
    for step in recipe.steps.find_all('step'):
        if ('has_minutes' in step.attrs) or ('has_hours' in step.attrs):
            recipes_time.append(recipe)
            break
len(recipes_time)

23469

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [21]:
recipes = pd.read_csv('data/recipes_sample.csv', parse_dates=['submitted'])

In [22]:
# сформируем словарь: id рецепта - ключ, количество шагов - значение
steps = {}
for recipe in steps_sample.recipes.find_all('recipe'):
    num = len(recipe.steps.find_all('step'))
    steps[int(recipe.id.text)] = num 
# из словаря сдклаем датафрейм
df_steps = pd.DataFrame(list(steps.items()), columns = ['id', 'num_steps'])
df_steps

,id,num_steps
0,44123,11
1,67664,3
2,38798,5
3,35173,7
4,84797,4
...,...,...
29995,267661,16
29996,386977,22
29997,103312,10
29998,486161,7


In [23]:
recipe = pd.merge(recipes, df_steps, left_on='id', right_on='id', how='left')
recipe['n_steps'] = recipe['n_steps'].fillna(recipe['num_steps'])
recipe.drop('num_steps', axis=1, inplace=True)
recipe

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,11.0,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,3.0,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,5.0,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,7.0,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,22.0,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,10.0,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,7.0,this is a delicious soup that i originally fou...,NaN


3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [24]:
if recipe['n_steps'].isnull().sum() == 0:
    print('Нет пропусков')
else:
    print('Есть пропуски')

Нет пропусков


In [25]:
recipe['n_steps'] = recipe['n_steps'].astype(int)
recipes.to_csv('data/recipes_sample_with_filled_nsteps.csv', index=False)